# Recolección de datos

In [5]:
import requests
import pandas as pd

## Obtención de distintos indicadores

In [3]:
def consultar(pais: str, indicador:str, pagina:int = 1):

    '''Función que contacta a la API y devuelve la respuesta
    que brinda la misma en formato json'''

    # Página de la api y path al recurso solicitado
    api_url = 'http://api.worldbank.org/v2/es'
    path = f'/country/{pais}/indicator/{indicador}'
    url = api_url + path

    # Creamos el diccionario con los parametros 
    # para el método get
    args= {
        "date":'1990:2020',
        'page':pagina,
        "per_page":1000,
        "format":"json",
        "prefix":"Getdata",
    }
   
    return requests.get(url,params=args).json()

In [4]:
def consultar_por_pais():
    '''Función que retorna un código de país,
    para, en caso que se quiera o se solicite,
    consultar en tiempo real la API'''
    paises=pd.read_parquet('../datasets/paises.parquet')

    seleccion = input("Ingrese un nombre de país: ")
    try:
        pais = paises.loc[paises.name.str.contains(seleccion, case=False),'id'].values[0]
    except:
        print("No se encontró el país, intente de nuevo")

In [217]:
# Estos indicadores fueron seleccionados 
# de forma manual al final del documento
indicadores = pd.read_csv('../datasets/indicadores_seleccionados.csv')

diccionario = { a:b for a,b in zip(indicadores.id.to_list(), 
          indicadores.name.to_list())}

In [95]:
def carga_incremental(pais, indicador):
    '''Función que a partir de un país y un indicador 
    llama a consultar y establece qué tipo de contenido tiene
    según eso devuelve o no un dataframe con todos los datos'''

    consulta = consultar(pais, indicador)
    try:
        # La primera parte de la respuesta nos indica en 
        # cuantas páginas de encuentra la información
        paginas = consulta[0]["pages"]

        # La segunda parte nos retorna una lista de 
        # diccionarios con la información que queríamos
        datos=consulta[1]

    except:
        print('No hay datos para:', indicador, pais)
        pass
    else:
        if paginas >= 1:
            # Agregamos los valores de las otras páginas a
            # nuestra lista de diccionarios
            for pagina in range(2,paginas+1):
                datos.extend(consultar(pais, indicador, pagina)[1])

            # Creo el DataFrame con todos los datos
            data = pd.json_normalize(datos)
            return data
        return pd.DataFrame(['error'],columns=['no_data'])


In [ ]:
for indicador in diccionario:
    print(indicador)
    datos = carga_incremental(pais='all', indicador=indicador)

    # Guardo el dataframe resultante
    datos.to_parquet(f'../datasets/df_TWB_{indicador}.parquet')
    print('Guardado')


# ETL

## Búsqueda de indicadores

La lista de indicadores bajo estudio se selecciona de forma manual a partir de los datos que indican distintas tesis de estudio. De todas formas los indicadores pasarán por un tamiz de análisis exploratorio

In [19]:
indicadores_todos = pd.read_csv('../datasets/indicadores.csv')

# Se completó los valores faltantes con 'na'
# para poder buscar sobre el dataframe
indicadores_todos.fillna('na',inplace=True)

In [37]:
interes = input("Indicador de interés: ")
indicadores_todos.loc[
     indicadores_todos.name.str.contains(interes,case=False),
     ['id','name','sourceNote']
     ]#.to_csv('temp.csv')

,id,name,sourceNote
17810,SP.POP.TOTL.FE.IN,"Población, mujeres","Población, mujeres"
17811,SP.POP.TOTL.FE.ZS,"Población, mujeres (% del total)",La población femenina es el porcentaje de la p...


In [26]:
interes = input("Indicador de interés: ")
seleccion = indicadores_todos.loc[indicadores_todos.id == interes,
     ['id','name','sourceNote']]
seleccion

,id,name,sourceNote
17815,SP.POP.TOTL.MA.ZS,"Población, hombres (% del total)",na


In [6]:
indicador_selec = pd.DataFrame()

In [108]:
indicador_selec = pd.concat([indicador_selec, seleccion], 
                            ignore_index=True)

## Exploración de los Datasets obtenidos

In [6]:
naciones_unidas = {
    "df_UNPD_mort_22" : "tasa_mortalidad_infantil",
    "df_UNPD_pop_54": "densidad_población_por_kilómetro_cuadrado)",
    "df_UNPD_imigrt_65": "migración_neta_total",
    "df_UNPD_pop_49": "población_total_por_sexo",
    "df_UNPD_mort_60": "total_muertes_por_sexo",
    "df_UNPD_pop_53": "tasa_bruta_cambio_natural_población",
    "df_UNPD_imigrt_66": "tasa_bruta_migración_neta",
    "df_UNPD_pop_72": "proporción_sexos_población_total",
    "df_UNPD_fam_1": "prevalencia_anticonceptivos_porcentaje",
    "df_UNPD_pop_67": "mediana_edad_población",
    "df_UNPD_mort_59": "tasa_bruta_mortalidad_por_1000_habitantes",
    "df_UNPD_pop_51": "tasa_bruta_variación_total_población",
    "df_UNPD_pop_50": "cambio_de_la_población",
    "df_UNPD_pop_41": "población_femenina_edad_reproductiva_(15-49 años)",
    "df_UNPD_mort_24": "tasa_mortalidad_menores_cinco_años",
    "df_UNPD_pop_52": "cambio_natural_población",
    "df_UNPD_fert_19": "tasa_fertilidad",
    "df_UNPD_marstat_42": "estado_civil_casado_porcentaje",
}

In [7]:
organizacion_salud = {'df_OMS_NUTRITION_ANAEMIA_CHILDREN_PREV': 'tasa_anemia_niños(%)',
'df_OMS_NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV': 'tasa_anemia_mujeres(%)',
'df_OMS_M_Est_cig_curr': 'tasa_consumo_cigarro(%)',
'df_OMS_SA_0000001688': 'tasa_consumo_alcohol(L)',
'df_OMS_NCD_BMI_30A' :'tasa_obesidad_pob(%)'}

In [8]:
banco_mundial = {
    'SP.DYN.LE00.IN': 'esperanza_vida_total',
    'SP.DYN.LE00.FE.IN': 'esperanza_vida_mujeres',
    'SP.DYN.LE00.MA.IN': 'esperanza_vida_varones',
    'SI.POV.GINI': 'índice_gini',
    'SE.XPD.TOTL.GD.ZS': 'gasto_púb_educacion_pje',
    'SE.COM.DURS': 'duración_educ_obligatoria',
    'NY.GDP.PCAP.CD': 'pib_pc_usd_actuales',
    'NY.GDP.MKTP.PP.CD': 'pib_ppa_prec_inter',
    'IQ.SCI.OVRL': 'capacidad_estadística',
    'SP.POP.TOTL.FE.ZS': 'población_mujeres_pje',
    'SP.POP.TOTL.MA.ZS': 'población_hombres_pje',
    'NY.GDP.PCAP.PP.CD': 'pib_pc_prec_inter',
    'AG.LND.FRST.ZS': 'porcentaje_de_bosque',
    'EN.ATM.CO2E.PC': 'emisiones_co2',
    'SH.XPD.CHEX.PC.CD': 'inversion_salud_percapita',
    'SH.MED.BEDS.ZS': 'camas_hospitales_c/1000personas',
    'SP.DYN.IMRT.IN': 'mortalidad_infantil_c/1000nacimientos',
    'SH.H2O.BASW.ZS': 'acceso_agua_potable(%)',
    'SH.STA.BASS.ZS': 'acceso_servicios_sanitarios(%)',
    'SH.STA.SUIC.P5': 'tasa_mortalidad_suicidio_c/100.000',
    'SL.UEM.TOTL.ZS': 'tasa_desempleo',
    'SP.URB.TOTL.IN.ZS': 'tasa_poblacion_urbana',
    'NY.GNP.PCAP.CD': 'INB_percapita'
}

In [9]:
problemas = ['df_UNPD_mort_22', 
            'df_UNPD_mort_24', 
            'df_UNPD_mort_60',
            'df_UNPD_pop_49']

In [10]:
import os
directorio = '../data_lake/'

with os.scandir(directorio) as ficheros:

    # Tomamos unicamente la fecha y el iso3 para usarlo como indice
    df_twb=pd.read_parquet('../data_lake/df_TWB_SP.DYN.LE00.IN.parquet')[['date','countryiso3code']]
    
    df_unpd = pd.read_parquet('../data_lake/df_TWB_SP.DYN.LE00.IN.parquet')[['date','countryiso3code']]
    df_unpd.set_index(['countryiso3code', 'date'], inplace=True)

    df_oms = pd.read_parquet('../data_lake/df_TWB_SP.DYN.LE00.IN.parquet')[['date','countryiso3code']]

    for fichero in ficheros:
        if fichero.name.startswith('df_TWB'):

            # obtengo el cógigo de indicador que se encuentra en el nombre del fichero
            codigo_fichero = fichero.name[7:-8]
            # busco el código en mi lista de códigos 
            # y procedo a renombrar la columna de interés
                
            df=pd.read_parquet(directorio+fichero.name)

            df_twb[banco_mundial[codigo_fichero]]=df.value
        
        elif fichero.name.startswith('df_UNPD'):
            codigo_fichero = fichero.name[:-8]
                
            if codigo_fichero in problemas:
                temp=pd.read_parquet(directorio+fichero.name)
                
                # Creo 3 tablas según el sexo sea hombre, mujer o ambos 
                # y selecciono las columnas de interés
                temp_male=temp.loc[temp.sex == "Male", ['iso3','timeLabel','value']]
                temp_female=temp.loc[temp.sex == "Female", ['iso3','timeLabel','value']]
                temp_both=temp.loc[temp.sex == "Both sexes", ['iso3','timeLabel','value']]

                # Renombro la columna de interés según el diccionario
                temp_both.rename(columns={"value":f"{naciones_unidas[codigo_fichero]}_ambos"}, inplace=True)
                temp_male.rename(columns={"value":f"{naciones_unidas[codigo_fichero]}_masc"}, inplace=True)
                temp_female.rename(columns={"value":f"{naciones_unidas[codigo_fichero]}_fem"}, inplace=True)

                # Asigno un index multiple
                temp_male.set_index(['iso3','timeLabel'], inplace=True)
                temp_female.set_index(['iso3','timeLabel'], inplace=True)
                temp_both.set_index(['iso3','timeLabel'], inplace=True)

                df_unpd = df_unpd.join(temp_both,
                    on=['countryiso3code','date']
                    )

                df_unpd = df_unpd.join(temp_male,
                    on=['countryiso3code','date']
                    )

                df_unpd = df_unpd.join(temp_female,
                    on=['countryiso3code','date']
                    )

            else:
                temp=pd.read_parquet(directorio+fichero.name)

                temp.set_index(['iso3','timeLabel'], inplace=True)

                temp.rename(columns={"value":naciones_unidas[codigo_fichero]}, inplace=True)
                
                df_unpd = df_unpd.join(temp[[naciones_unidas[codigo_fichero]]], 
                                        on=['countryiso3code','date'])

Al usar como tabla base una tabla con más valores de los que poseían las tablas del banco mundial, nos queda una tabla con valores faltantes en todas sus filas. Esas filas con todos los datos faltantes son las que se procede a eliminar a continuación

In [7]:
df_unpd

densidad_población_por_kilómetro_cuadrado)  \
countryiso3code date                                               
AFE             2020                                         NaN   
                2019                                         NaN   
                2018                                         NaN   
                2017                                         NaN   
                2016                                         NaN   
...                                                          ...   
ZWE             1994                                   28.069262   
                1993                                   27.904661   
                1992                                   27.508081   
                1991                                   26.826457   
                1990                                   26.144224   

                      tasa_mortalidad_infantil_ambos  \
countryiso3code date                                   
AFE             2020                             NaN   
                2019                             NaN   
                2018                             NaN   
                2017                             NaN   
                2016                             NaN   
...                                              ...   
ZWE             1994                        58.92822   
                1993                        57.05718   
                1992                        54.79743   
                1991                        52.62272   
                1990                        50.91590   

                      tasa_mortalidad_infantil_masc  \
countryiso3code date                                  
AFE             2020                            NaN   
                2019                            NaN   
                2018                            NaN   
                2017                            NaN   
                2016                            NaN   
...                                             ...   
ZWE             1994                       62.56177   
                1993                       60.65919   
                1992                       58.37067   
                1991                       56.15472   
                1990                       54.46490   

                      tasa_mortalidad_infantil_fem  migración_neta_total  \
countryiso3code date                                                       
AFE             2020                           NaN                   NaN   
                2019                           NaN                   NaN   
                2018                           NaN                   NaN   
                2017                           NaN                   NaN   
                2016                           NaN                   NaN   
...                                            ...                   ...   
ZWE             1994                      55.38732             -144401.0   
                1993                      53.53403             -201588.0   
                1992                      51.28329                8095.0   
                1991                      49.13535                 998.0   
                1990                      47.40078                2355.0   

                      proporción_sexos_población_total  \
countryiso3code date                                     
AFE             2020                               NaN   
                2019                               NaN   
                2018                               NaN   
                2017                               NaN   
                2016                               NaN   
...                                                ...   
ZWE             1994                         90.884607   
                1993                         91.347296   
                1992                         92.103078   
                1991                         92.603579   
       

In [11]:
df_unpd.dropna(how='all',inplace=True)

In [12]:
df_unpd

densidad_población_por_kilómetro_cuadrado)  \
countryiso3code date                                               
AFG             2020                                   60.022286   
                2019                                   58.169923   
                2018                                   56.502403   
                2017                                   54.895483   
                2016                                   53.344249   
...                                                          ...   
ZWE             1994                                   28.069262   
                1993                                   27.904661   
                1992                                   27.508081   
                1991                                   26.826457   
                1990                                   26.144224   

                      tasa_mortalidad_infantil_ambos  \
countryiso3code date                                   
AFG             2020                        45.76647   
                2019                        46.71307   
                2018                        47.83544   
                2017                        49.37693   
                2016                        52.45424   
...                                              ...   
ZWE             1994                        58.92822   
                1993                        57.05718   
                1992                        54.79743   
                1991                        52.62272   
                1990                        50.91590   

                      tasa_mortalidad_infantil_masc  \
countryiso3code date                                  
AFG             2020                       49.31858   
                2019                       50.34222   
                2018                       51.49845   
                2017                       53.10363   
                2016                       56.34064   
...                                             ...   
ZWE             1994                       62.56177   
                1993                       60.65919   
                1992                       58.37067   
                1991                       56.15472   
                1990                       54.46490   

                      tasa_mortalidad_infantil_fem  migración_neta_total  \
countryiso3code date                                                       
AFG             2020                      42.05730              166821.0   
                2019                      42.92233               -8082.0   
                2018                      44.00820              -47205.0   
                2017                      45.48193              -47090.0   
                2016                      48.39276              -90238.0   
...                                            ...                   ...   
ZWE             1994                      55.38732             -144401.0   
                1993                      53.53403             -201588.0   
                1992                      51.28329                8095.0   
                1991                      49.13535                 998.0   
                1990                      47.40078                2355.0   

                      proporción_sexos_población_total  \
countryiso3code date                                     
AFG             2020                        102.138859   
                2019                        102.202032   
                2018                        102.276798   
                2017                        102.350156   
                2016                        102.369303   
...                                                ...   
ZWE             1994                         90.884607   
                1993                         91.347296   
                1992                         92.103078   
                1991                         92.603579   
       

In [21]:
#df_unpd.to_parquet('../data_ware_house/df_unpd_todos.parquet')

In [13]:
df_twb.set_index(['countryiso3code','date'], inplace=True)

In [11]:
df_twb

acceso_agua_potable(%)  porcentaje_de_bosque  \
countryiso3code date                                                 
AFE             2020               59.511910             30.174186   
                2019               58.782401             30.391558   
                2018               57.860301             30.611444   
                2017               56.921012             30.824248   
                2016               55.963351             31.039613   
...                                      ...                   ...   
ZWE             1994                     NaN             48.190255   
                1993                     NaN             48.309345   
                1992                     NaN             48.428435   
                1991                     NaN             48.547525   
                1990                     NaN             48.666615   

                      tasa_mortalidad_suicidio_c/100.000  \
countryiso3code date                                       
AFE             2020                                 NaN   
                2019                            8.036293   
                2018                            8.128884   
                2017                            8.244062   
                2016                            8.324659   
...                                                  ...   
ZWE             1994                                 NaN   
                1993                                 NaN   
                1992                                 NaN   
                1991                                 NaN   
                1990                                 NaN   

                      mortalidad_infantil_c/1000nacimientos  \
countryiso3code date                                          
AFE             2020                              42.447198   
                2019                              43.591252   
                2018                              44.795221   
                2017                              46.108140   
                2016                              47.527244   
...                                                     ...   
ZWE             1994                              55.000000   
                1993                              53.700000   
                1992                              52.400000   
                1991                              51.200000   
                1990                              50.400000   

                      pib_ppa_prec_inter  población_mujeres_pje  \
countryiso3code date                                              
AFE             2020        2.478218e+12              50.364579   
                2019        2.517975e+12              50.373008   
                2018        2.418685e+12              50.382358   
                2017        2.299420e+12              50.392615   
                2016        2.212683e+12              50.403727   
...                                  ...                    ...   
ZWE             1994        2.165871e+10              50.486345   
                1993        1.941304e+10              50.412002   
                1992        1.876622e+10              50.343007   
                1991        2.016618e+10              50.285057   
                1990        1.848402e+10              50.242144   

                      emisiones_co2  tasa_poblacion_urbana  \
countryiso3code date                                         
AFE             2020            NaN              36.783306   
                2019       0.913618              36.291322   
                2018       0.917507              35.807770   
                2017       0.937926              35.332373   
                2016       0.948410              34.865101   
...                             ...                    ...   
ZWE             1994       1.417186              31.335000   
                1993       1.539741              30.940000   
                19

## Sección del desarrollador

In [33]:
select = 'df_UNPD_pop_49.parquet'
temp = pd.read_parquet(f'../data_lake/{select}')

#temp_male=temp.loc[temp.sex == "Male", ['iso3','timeLabel','value']]

temp
#temp.set_index(['iso3','timeLabel'], inplace=True)

#temp.Dim1.value_counts()

,locationId,location,iso3,iso2,locationTypeId,indicatorId,indicator,indicatorDisplayName,sourceId,source,...,estimateMethodId,estimateMethod,sexId,sex,ageId,ageLabel,ageStart,ageEnd,ageMid,value
0,4,Afghanistan,AFG,AF,4,49,Total population by sex,Total population by sex,25,World Population Prospects,...,2,Interpolation,1,Male,188,Total,0,-1,0,5348387
1,4,Afghanistan,AFG,AF,4,49,Total population by sex,Total population by sex,25,World Population Prospects,...,2,Interpolation,2,Female,188,Total,0,-1,0,5346409
2,4,Afghanistan,AFG,AF,4,49,Total population by sex,Total population by sex,25,World Population Prospects,...,2,Interpolation,3,Both sexes,188,Total,0,-1,0,10694796
3,4,Afghanistan,AFG,AF,4,49,Total population by sex,Total population by sex,25,World Population Prospects,...,2,Interpolation,1,Male,188,Total,0,-1,0,5372960
4,4,Afghanistan,AFG,AF,4,49,Total population by sex,Total population by sex,25,World Population Prospects,...,2,Interpolation,2,Female,188,Total,0,-1,0,5372208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21013,894,Zambia,ZMB,ZM,4,49,Total population by sex,Total population by sex,25,World Population Prospects,...,2,Interpolation,2,Female,188,Total,0,-1,0,9314079
21014,894,Zambia,ZMB,ZM,4,49,Total population by sex,Total population by sex,25,World Population Prospects,...,2,Interpolation,3,Both sexes,188,Total,0,-1,0,18380477
21015,894,Zambia,ZMB,ZM,4,49,Total population by sex,Total population by sex,25,World Population Prospects,...,2,Interpolation,1,Male,188,Total,0,-1,0,9338614
21016,894,Zambia,ZMB,ZM,4,49,Total population by sex,Total population by sex,25,World Population Prospects,...,2,Interpolation,2,Female,188,Total,0,-1,0,9589102


In [ ]:
# Creo 3 tablas según el sexo sea hombre, mujer o ambos 
# y selecciono las columnas de interés
temp_severa=temp.loc[temp.sex == "SEVERE", ['SpatialDim','timeLabel','NumericValue']]
temp_moderada=temp.loc[temp.sex == "MODERATE", ['SpatialDim','timeLabel','NumericValue']]
temp_leve=temp.loc[temp.sex == "MILD", ['SpatialDim','timeLabel','NumericValue']]

# Renombro la columna de interés según el diccionario
temp_leve.rename(
    columns={"NumericValue":organizacion_salud[codigo_fichero]}, 
    inplace=True)
temp_severa.rename(
    columns={"NumericValue":organizacion_salud[codigo_fichero]}, 
    inplace=True)
temp_moderada.rename(
    columns={"NumericValue":organizacion_salud[codigo_fichero]}, 
    inplace=True)

# Asigno un index multiple
temp_severa.set_index(['SpatialDim','timeLabel'], inplace=True)
temp_moderada.set_index(['SpatialDim','timeLabel'], inplace=True)
temp_leve.set_index(['SpatialDim','timeLabel'], inplace=True)

df_oms = df_oms.join(temp_leve,
    on=['countryiso3code','date']
    )

df_oms = df_oms.join(temp_severa,
    on=['countryiso3code','date']
    )

df_oms = df_oms.join(temp_moderada,
    on=['countryiso3code','date']
    )

# Clasificación según nivel de ingreso per cápita

In [14]:
df_twb['nivel_ingreso'] = pd.cut(df_twb['INB_percapita'],
                bins=[0,1025,3995,12375,121901],
                labels=['low_income', 
                        'lower_middle_income', 
                        'upper_middle_income',
                        'high_income'],
                include_lowest = True)

# Analisis exploratorio

Al ser nuestro objetivo la esperanza de vida, no sería recomendable inventar esos datos con un promedio, ni nada parecido. por lo que vamos a limpiar todas las filas que tengan valores nulos en la esperanza de vida al nacer

In [15]:
df_twb.dropna(subset=['esperanza_vida_total'], inplace=True)

In [23]:
#df_twb.to_parquet('../data_ware_house/df_twb_todos.parquet')

Verificamos nuevamente los datos

In [16]:
df_twb.isna().sum()

acceso_agua_potable(%)                   2575
porcentaje_de_bosque                      273
tasa_mortalidad_suicidio_c/100.000       3049
mortalidad_infantil_c/1000nacimientos     473
pib_ppa_prec_inter                        651
población_mujeres_pje                     204
emisiones_co2                             732
tasa_poblacion_urbana                      71
inversion_salud_percapita                3150
esperanza_vida_total                        0
tasa_desempleo                            626
población_hombres_pje                     204
INB_percapita                             730
duración_educ_obligatoria                2651
índice_gini                              5937
esperanza_vida_mujeres                      0
esperanza_vida_varones                      0
gasto_púb_educacion_pje                  3368
acceso_servicios_sanitarios(%)           2583
capacidad_estadística                    4761
camas_hospitales_c/1000personas          4049
pib_pc_prec_inter                 

Vamos a eliminar todas aquellas columnas que tengan más del 35% de datos núlos. Dado que de trabajar sobre ellas con alguna estratégia de imputación no sería adecuado.

In [21]:
(df_twb.isna().sum()/df_twb.shape[0])*100

acceso_agua_potable(%)                   33.576738
mortalidad_infantil_c/1000nacimientos     6.167688
emisiones_co2                             9.544921
tasa_poblacion_urbana                     0.925805
esperanza_vida_total                      0.000000
INB_percapita                             9.518842
esperanza_vida_mujeres                    0.000000
esperanza_vida_varones                    0.000000
acceso_servicios_sanitarios(%)           33.681054
pib_pc_prec_inter                         8.527839
pib_pc_usd_actuales                       4.276959
nivel_ingreso                             9.518842
dtype: float64

In [18]:
df_twb.drop(columns=[   'tasa_mortalidad_suicidio_c/100.000',
                        'inversion_salud_percapita',
                        'índice_gini',
                        'capacidad_estadística', 
                        'gasto_púb_educacion_pje', 
                        'duración_educ_obligatoria',
                        'camas_hospitales_c/1000personas'],
            inplace=True
            )

In [17]:
corr_matrix = df_twb.corr()
corr_matrix["esperanza_vida_total"].sort_values(ascending=False)

esperanza_vida_total                     1.000000
esperanza_vida_mujeres                   0.994658
esperanza_vida_varones                   0.993956
acceso_servicios_sanitarios(%)           0.863164
acceso_agua_potable(%)                   0.851376
tasa_poblacion_urbana                    0.663362
pib_pc_prec_inter                        0.608063
INB_percapita                            0.590319
pib_pc_usd_actuales                      0.570186
emisiones_co2                            0.523546
pib_ppa_prec_inter                       0.143315
población_hombres_pje                    0.052518
tasa_desempleo                           0.034451
porcentaje_de_bosque                    -0.011521
población_mujeres_pje                   -0.052518
mortalidad_infantil_c/1000nacimientos   -0.930248
Name: esperanza_vida_total, dtype: float64

Dividimos el dataset de trabajo en cuatro datasets para analizar según su nivel de ingresos

In [18]:
df_twb['nivel_ingreso'].value_counts()

lower_middle_income    2081
low_income             1805
high_income            1603
upper_middle_income    1450
Name: nivel_ingreso, dtype: int64

In [19]:
ingreso_bajo = df_twb.loc[df_twb['nivel_ingreso'] == 'low_income']
ingreso_medio_bajo = df_twb.loc[df_twb['nivel_ingreso'] == 'lower_middle_income']
ingreso_medio_alto = df_twb.loc[df_twb['nivel_ingreso'] == 'upper_middle_income']
ingreso_alto = df_twb.loc[df_twb['nivel_ingreso'] == 'high_income']


In [22]:
(ingreso_alto.isnull().sum()/1603)*100

acceso_agua_potable(%)                   24.703681
porcentaje_de_bosque                      4.865876
mortalidad_infantil_c/1000nacimientos    11.977542
pib_ppa_prec_inter                        4.179663
población_mujeres_pje                     3.119152
emisiones_co2                            14.784779
tasa_poblacion_urbana                     0.000000
esperanza_vida_total                      0.000000
tasa_desempleo                            8.484092
población_hombres_pje                     3.119152
INB_percapita                             0.000000
esperanza_vida_mujeres                    0.000000
esperanza_vida_varones                    0.000000
acceso_servicios_sanitarios(%)           24.017467
pib_pc_prec_inter                         4.179663
pib_pc_usd_actuales                       0.000000
nivel_ingreso                             0.000000
dtype: float64

Descubrimos que la calidad del dato mejora casi todos los indicadores, excepto en la emisiones de co2 y la mortalidad infantil que empeora. Tal vez indique que los países de altos ingresos no se encuentran interesados en hacerle un seguimiento a esa información

In [23]:
corr_matrix = ingreso_alto.corr()
corr_matrix["esperanza_vida_total"].sort_values(ascending=False)

esperanza_vida_total                     1.000000
esperanza_vida_varones                   0.973058
esperanza_vida_mujeres                   0.972080
acceso_servicios_sanitarios(%)           0.612380
acceso_agua_potable(%)                   0.608931
INB_percapita                            0.526020
pib_pc_usd_actuales                      0.502948
pib_pc_prec_inter                        0.281642
tasa_poblacion_urbana                    0.274005
población_mujeres_pje                    0.144334
pib_ppa_prec_inter                       0.048801
porcentaje_de_bosque                    -0.018948
tasa_desempleo                          -0.133377
población_hombres_pje                   -0.144334
emisiones_co2                           -0.173597
mortalidad_infantil_c/1000nacimientos   -0.758128
Name: esperanza_vida_total, dtype: float64

Descubrimos que el impacto que tenían ciertos indicadores en la esperanza de vida al nacer bajaron drásticamente en este grupo. Vamos a analizar los grupos siguientes.

In [22]:
corr_matrix = ingreso_medio_alto.corr()
corr_matrix["esperanza_vida_total"].sort_values(ascending=False)

esperanza_vida_total                     1.000000
esperanza_vida_mujeres                   0.973269
esperanza_vida_varones                   0.973164
acceso_agua_potable(%)                   0.667850
acceso_servicios_sanitarios(%)           0.667396
INB_percapita                            0.206262
tasa_poblacion_urbana                    0.171759
pib_pc_usd_actuales                      0.146698
pib_pc_prec_inter                        0.076660
población_mujeres_pje                    0.025265
pib_ppa_prec_inter                      -0.021486
población_hombres_pje                   -0.025265
emisiones_co2                           -0.072779
porcentaje_de_bosque                    -0.122549
tasa_desempleo                          -0.292788
mortalidad_infantil_c/1000nacimientos   -0.791533
Name: esperanza_vida_total, dtype: float64

In [23]:
corr_matrix = ingreso_medio_bajo.corr()
corr_matrix["esperanza_vida_total"].sort_values(ascending=False)

esperanza_vida_total                     1.000000
esperanza_vida_mujeres                   0.982985
esperanza_vida_varones                   0.981386
acceso_agua_potable(%)                   0.728386
acceso_servicios_sanitarios(%)           0.706355
pib_pc_prec_inter                        0.351017
INB_percapita                            0.346589
tasa_poblacion_urbana                    0.344056
pib_pc_usd_actuales                      0.303978
emisiones_co2                            0.229484
población_hombres_pje                    0.091574
porcentaje_de_bosque                     0.039084
pib_ppa_prec_inter                       0.034453
población_mujeres_pje                   -0.091574
tasa_desempleo                          -0.122231
mortalidad_infantil_c/1000nacimientos   -0.831776
Name: esperanza_vida_total, dtype: float64

In [24]:
corr_matrix = ingreso_bajo.corr()
corr_matrix["esperanza_vida_total"].sort_values(ascending=False)

esperanza_vida_total                     1.000000
esperanza_vida_mujeres                   0.993737
esperanza_vida_varones                   0.992926
acceso_agua_potable(%)                   0.589490
acceso_servicios_sanitarios(%)           0.551094
pib_pc_prec_inter                        0.489476
INB_percapita                            0.450990
pib_pc_usd_actuales                      0.391872
emisiones_co2                            0.365573
tasa_poblacion_urbana                    0.256688
pib_ppa_prec_inter                       0.197939
población_hombres_pje                    0.151878
tasa_desempleo                           0.075077
porcentaje_de_bosque                    -0.088007
población_mujeres_pje                   -0.151878
mortalidad_infantil_c/1000nacimientos   -0.854560
Name: esperanza_vida_total, dtype: float64

Podemos eliminar tambien, de forma general aquellas características que tenían una correlación debil desde el inicio y que se han mantenido igual desde el principio. Descubrimos también que la hipotesis de la tesis resultó ser cierta. A menor ingreso, el impacto del PIB resulta ser mayor en la esperanza de vida al nacer

In [20]:
df_twb.drop(columns=[   'pib_ppa_prec_inter',
                        'población_hombres_pje',
                        'tasa_desempleo',
                        'porcentaje_de_bosque',
                        'población_mujeres_pje'
                        ],
            inplace=True
            )

In [25]:
corr_matrix = df_twb.corr()
corr_matrix

,acceso_agua_potable(%),mortalidad_infantil_c/1000nacimientos,emisiones_co2,tasa_poblacion_urbana,esperanza_vida_total,INB_percapita,esperanza_vida_mujeres,esperanza_vida_varones,acceso_servicios_sanitarios(%),pib_pc_prec_inter,pib_pc_usd_actuales
acceso_agua_potable(%),1.000000,-0.864287,0.522210,0.646636,0.851376,0.478627,0.859703,0.833330,0.893523,0.540016,0.461338
mortalidad_infantil_c/1000nacimientos,-0.864287,1.000000,-0.513802,-0.643202,-0.930248,-0.515410,-0.938490,-0.910665,-0.871028,-0.552285,-0.506229
emisiones_co2,0.522210,-0.513802,1.000000,0.625685,0.523546,0.648021,0.521818,0.523698,0.586259,0.812946,0.573790
tasa_poblacion_urbana,0.646636,-0.643202,0.625685,1.000000,0.663362,0.561574,0.670932,0.648770,0.661373,0.663245,0.497955
esperanza_vida_total,0.851376,-0.930248,0.523546,0.663362,1.000000,0.590319,0.994658,0.993956,0.863164,0.608063,0.570186
INB_percapita,0.478627,-0.515410,0.648021,0.561574,0.590319,1.000000,0.575770,0.599912,0.524962,0.880205,0.990831
esperanza_vida_mujeres,0.859703,-0.938490,0.521818,0.670932,0.994658,0.575770,1.000000,0.977419,0.878486,0.591047,0.555283
esperanza_vida_varones,0.833330,-0.910665,0.523698,0.648770,0.993956,0.599912,0.977419,1.000000,0.838080,0.621900,0.580141
acceso_servicios_sanitarios(%),0.893523,-0.871028,0.586259,0.661373,0.863164,0.524962,0.878486,0.838080,1.000000,0.595713,0.504653
pib_pc_prec_inter,0.540016,-0.552285,0.812946,0.663245,0.608063,0.880205,0.591047,0.621900,0.595713,1.000000,0.885277


Una vez hecho esto nos regresamos a los bloques anteriores para ejecutarlos ahora sin las columas desechadas

In [73]:
idx=pd.IndexSlice
analisis = ingreso_bajo.loc[idx['AFE', :, :],:]
analisis

acceso_agua_potable(%)  \
countryiso3code date                           
AFE             2004               44.192555   
                2003               43.359634   
                2002               42.548500   
                2001               41.735135   
                2000               40.931834   
                1999                     NaN   
                1998                     NaN   
                1997                     NaN   
                1996                     NaN   
                1995                     NaN   
                1994                     NaN   
                1993                     NaN   
                1992                     NaN   
                1991                     NaN   
                1990                     NaN   

                      mortalidad_infantil_c/1000nacimientos  emisiones_co2  \
countryiso3code date                                                         
AFE             2004                              73.220770       1.036161   
                2003                              76.243386       0.991013   
                2002                              79.363100       0.964187   
                2001                              82.531884       0.962371   
                2000                              85.557131       0.894017   
                1999                              88.289462       0.904871   
                1998                              90.786908       0.967182   
                1997                              93.015988       0.967836   
                1996                              94.979329       0.947393   
                1995                              96.637270       0.936570   
                1994                              99.365522       0.917093   
                1993                              99.333634       0.916485   
                1992                             100.310805       0.915661   
                1991                             101.351628       0.953224   
                1990                             102.393986       0.999038   

                      tasa_poblacion_urbana  esperanza_vida_total  \
countryiso3code date                                                
AFE             2004              29.961824             53.228913   
                2003              29.631273             52.585853   
                2002              29.304199             52.043149   
                2001              28.977074             51.606458   
                2000              28.669286             51.276129   
                1999              28.384542             51.044191   
                1998              28.105740             50.897607   
                1997              27.819697             50.820614   
                1996              27.505942             50.796157   
                1995              27.169617             50.808484   
                1994              26.812288             50.841865   
                1993              26.413930             50.890246   
                1992              25.920719             50.957262   
                1991              25.408236             51.048414   
                1990              24.919224             51.154113   

                      INB_percapita  esperanza_vida_mujeres  \
countryiso3code date                                          
AFE             2004     820.910265               55.041854   
                2003     674.488409               54.438284   
                2002     624.363657               53.944809   
                2001     639.670444               53.564031   
                2000     666.738458               53.291143   
                1999     671.949572               53.114287   
                1998     693.821318               53.017289   
                1997     751.766497               52.979222   
                1996     748.766205               52.980451   
           

In [24]:
import plotly.express as px

fig = px.line(df_twb, 
                x=pd.to_datetime(df_twb.index.get_level_values('date')), 
                y="esperanza_vida_total", 
                title='Esperanza de vida en todos los países',
                color= df_twb.index.get_level_values('countryiso3code'))
fig.show()

In [77]:
ingreso_bajo.reset_index(inplace=True)
ingreso_medio_bajo.reset_index(inplace=True)
ingreso_medio_alto.reset_index(inplace=True)
ingreso_alto.reset_index(inplace=True)

In [81]:
prom_ingreso_bajo = ingreso_bajo.groupby('date').mean()
prom_ingreso_medio_bajo = ingreso_medio_bajo.groupby('date').mean()
prom_ingreso_medio_alto = ingreso_medio_alto.groupby('date').mean()
prom_ingreso_alto = ingreso_alto.groupby('date').mean()

In [102]:
promedio = df_twb.groupby(['nivel_ingreso','date']).mean()

In [103]:
promedio.reset_index(inplace=True)

In [106]:
import plotly.express as px

fig = px.line(promedio, 
                x='date', 
                y="esperanza_vida_total", 
                title='Esperanza de vida agrupado por nivel de ingreso',
                color= 'nivel_ingreso')
fig.show()

In [99]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=prom_ingreso_bajo.index, 
                y=prom_ingreso_bajo.esperanza_vida_total,
                name="Promedio de esperanza de vida, bajos ingreso"
))
fig.add_trace(go.Scatter(x=prom_ingreso_alto.index, 
                y=prom_ingreso_alto.esperanza_vida_total,
                name="Promedio de esperanza de vida, altos ingreso"
))


fig.show()